In [1]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

from plotly import express as px
from typing import List, Tuple
import pickle
import numpy as np

2025-08-20 14:06:53.031841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-20 14:06:53.138661: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-20 14:06:54.267118: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/

In [2]:
import sys
sys.path.append('../../../../')

## load

In [3]:
with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto_and_coord/folds_training_dicts.pkl', 'rb') as f:
    folds_training_dicts = pickle.load(f)

2025-08-20 14:06:57.263441: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
fold = folds_training_dicts[0]
fold.keys()

dict_keys(['sizes_train', 'components_train', 'num_patches_train', 'uniprots_train', 'labels_train', 'sizes_validation', 'components_validation', 'num_patches_validation', 'uniprots_validation', 'labels_validation', 'sizes_test', 'components_test', 'num_patches_test', 'uniprots_test', 'labels_test', 'coordinates_train', 'coordinates_validation', 'coordinates_test'])

In [5]:
input_data = fold['components_train']
coordinates = fold['coordinates_train']
size_value = fold['sizes_train']
n_patches_hot_encoded_value = fold['num_patches_train']
max_number_of_patches = 10

## debug pairwise transition

In [6]:
from model import build_model, create_masked_inputs

In [7]:
model = build_model(features_mlp_hidden_sizes=[[32, 32]],
            features_mlp_dropout_rate=0.2,
            output_mlp_hidden_sizes=[[32, 32]], output_mlp_dropout_rate=0.2,
            attention_mlp_hidden_sizes=[[32, 32]], attention_mlp_dropout_rate=0.2,
            activation='relu', input_shape=(10, 9), max_number_of_patches=10, attention_dimension=32,
            gaussian_xrange=(0, 100),
            pairs_channel_dimension=32, num_heads=8, use_pair_bias=True)

initializing projection for skip connection: 11 -> 32


In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 number_of_patches_input (Input  [(None, 11)]        0           []                               
 Layer)                                                                                           
                                                                                                  
 features_input (InputLayer)    [(None, 10, 9)]      0           []                               
                                                                                                  
 tf.math.argmax (TFOpLambda)    (None,)              0           ['number_of_patches_input[0][0]']
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 1)           0           ['tf.math.argmax[0][0]']     

In [9]:
batch_size = 7

In [11]:
model([input_data[:batch_size], coordinates[:batch_size], size_value[:batch_size], n_patches_hot_encoded_value[:batch_size]])

<tf.Tensor: shape=(7, 1), dtype=float32, numpy=
array([[0.7819127 ],
       [0.7979585 ],
       [0.7717895 ],
       [0.80579376],
       [0.7794672 ],
       [0.79469633],
       [0.7478455 ]], dtype=float32)>